In [ ]:
#| default_exp components

# Components

In [ ]:
#| export
from html.parser import HTMLParser
from dataclasses import dataclass, asdict, is_dataclass, make_dataclass, replace, astuple, MISSING

from fastcore.utils import *
from fastcore.xml import *
from fastcore.meta import use_kwargs, delegates

try: from IPython import display
except ImportError: display=None

In [ ]:
from lxml import html as lx
from pprint import pprint

In [ ]:
#| export
def show(xt,*rest):
    if rest: xt = (xt,)+rest
    return display.HTML(to_xml(xt))

In [ ]:
#| export
named = set('a button form frame iframe img input map meta object param select textarea'.split())
html_attrs = 'id cls title style accesskey contenteditable dir draggable enterkeyhint hidden inert inputmode lang popover spellcheck tabindex translate'.split()
hx_attrs = 'get post put delete patch trigger target swap include select indicator push_url confirm disable replace_url on'
hx_attrs = html_attrs + [f'hx_{o}' for o in hx_attrs.split()]

In [ ]:
#| export
def xt_html(tag: str, *c, id=None, cls=None, title=None, style=None, **kwargs):
    kwargs['id'],kwargs['cls'],kwargs['title'],kwargs['style'] = id,cls,title,style
    tag,c,kw = xt(tag, *c, **kwargs)
    if tag in named and 'id' in kw and 'name' not in kw: kw['name'] = kw['id']
    return XT(tag,c,kw)

In [ ]:
#| export
@use_kwargs(hx_attrs, keep=True)
def xt_hx(tag: str, *c, target_id=None, **kwargs):
    if target_id: kwargs['hx_target'] = '#'+target_id
    return xt_html(tag, *c, **kwargs)

In [ ]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details', 'Dialog',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt_hx, o.lower())

In [ ]:
#| hide
both_attrs = html_attrs+hx_attrs

with open('../fasthtml/components.pyi', 'w') as f:
    attrs_str = ', '.join(f'{t}:str|None=None' for t in both_attrs)
    f.write(f"def xt_hx(tag: str, *c, {attrs_str}, **kwargs): ...\n")
    for o in _all_:
        attrs = (['name'] if o.lower() in named else []) + both_attrs
        attrs_str = ', '.join(f'{t}:str|None=None' for t in attrs)
        f.write(f"def {o}(*c, {attrs_str}, **kwargs): ...\n")

For tags that have a `name` attribute, it will be set to the value of `id` if not provided explicitly:

In [ ]:
Form(Button(target_id='foo', id='btn'),
     hx_post='/', target_id='tgt', id='frm')

```xml
<form hx-post="/" hx-target="#tgt" id="frm" name="frm">
  <button hx-target="#foo" id="btn" name="btn"></button>
</form>

```

In [ ]:
#| export
def _fill_item(item, obj):
    if not isinstance(item,list): return item
    tag,cs,attr = item
    if isinstance(cs,tuple): cs = tuple(_fill_item(o, obj) for o in cs)
    name = attr.get('name', None)
    val = None if name is None else obj.get(name, None)
    if val is not None:
        if tag=='input':
            if attr.get('type', '') in ('checkbox','radio'):
                if val: attr['checked'] = '1'
                else: attr.pop('checked', '')
            else: attr['value'] = val
        if tag=='textarea': cs=(val,)
    return XT(tag,cs,attr)

In [ ]:
#| export
def fill_form(form:XT, obj)->XT:
    "Fills named items in `form` using attributes in `obj`"
    if not isinstance(obj,dict): obj = asdict(obj)
    return _fill_item(form, obj)

In [ ]:
@dataclass
class TodoItem:
    title:str; id:int; done:bool; details:str
                
todo = TodoItem(id=2, title="Profit", done=True, details="Details")
check = Label(Input(type="checkbox", id="done"), "Done")
form = Form(Fieldset(Input(id="title"), check, Input(type="hidden", id="id"), Textarea(id='details'), Button("Save")))
form = fill_form(form, todo)
form

```xml
<form>
  <fieldset>
    <input id="title" name="title" value="Profit">
    <label>
      <input type="checkbox" id="done" name="done" checked="1">
Done
    </label>
    <input type="hidden" id="id" name="id" value="2">
    <textarea id="details" name="details">
Details
    </textarea>
    <button>
Save
    </button>
  </fieldset>
</form>

```

In [ ]:
#|export
def fill_dataclass(src, dest):
    "Modifies dataclass in-place and returns it"
    for nm,val in asdict(src).items(): setattr(dest, nm, val)
    return dest

In [ ]:
nt = TodoItem('', 0, False, '')
fill_dataclass(todo, nt)
nt

TodoItem(title='Profit', id=2, done=True, details='Details')

In [ ]:
#| exports
def find_inputs(e, tags='input', **kw):
    # Recursively find all elements in `e` with `tags` and attrs matching `kw`
    if not isinstance(e, (list,tuple)): return []
    inputs = []
    if isinstance(tags,str): tags = [tags]
    elif tags is None: tags = []
    cs = e
    if isinstance(e, list):
        tag,cs,attr = e
        if e[0] in tags and kw.items()<=e[2].items(): inputs.append(e)
    for o in cs: inputs += find_inputs(o, tags, **kw)
    return inputs

In [ ]:
find_inputs(form, id='title')

[['input', (), {'id': 'title', 'name': 'title', 'value': 'Profit'}]]

You can also use lxml for more sophisticated searching:

In [ ]:
elem = lx.fromstring(to_xml(form))
elem.xpath("//input[@id='title']/@value")

['Profit']

In [ ]:
#| export
def __getattr__(tag):
    if tag.startswith('_') or tag[0].islower(): raise AttributeError
    def _f(*c, target_id=None, **kwargs): return xt_hx(tag, *c, target_id=target_id, **kwargs)
    return _f

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()